In [1]:
using Pkg

# Pkg.add("HTTP")
# Pkg.add("JSON")

# Pkg.add("LightXML")
# Pkg.add("XML")
#Pkg.add("EzXML")

using EzXML
import HTTP
import JSON
using DataFrames


In [59]:
# login credentials
u = "Kenneth.Rosal"
p = "Roadsafe093"
url = "https://infoconnect1.highwayinfo.govt.nz:443/ic/jbi/TrafficCameras2/REST/FeedService/"
credentials = Dict("username" => u, "password" =>p)

Dict{String, String} with 2 entries:
  "password" => "Roadsafe093"
  "username" => "Kenneth.Rosal"

In [60]:
# HTTP.get(url)
res = HTTP.get(url,credentials)


HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Date: Sat, 15 Oct 2022 02:06:29 GMT
Server: Apache
Strict-Transport-Security: max-age=63072000; includeSubdomains; preload
Set-Cookie: JSESSIONID=FCD2B63C801531EB0031B9792949B2AB.jvm1; Path=/ic; Secure
Content-Type: text/xml
Transfer-Encoding: chunked

<?xml version='1.0' encoding='UTF-8'?><tns:getCamerasResponse xmlns:tns="https://infoconnect.highwayinfo.govt.nz/schemas/camera2"><tns:camera><tns:description>South along Hinds Highway from Lagmhor Rd</tns:description><tns:direction>Southbound</tns:direction><tns:group>NA</tns:group><tns:id>714</tns:id><tns:imageUrl>http://www.trafficnz.info/camera/714.jpg</tns:imageUrl><tns:lat>-43.919632</tns:lat><tns:lon>171.721055</tns:lon><tns:name>SH1 Tinwald</tns:name><tns:offline>false</tns:offline><tns:region>Ashburton</tns:region><tns:thumbUrl>http://www.trafficnz.info/camera/thumb/714.jpg</tns:thumbUrl><tns:underMaintenance>false</tns:underMaintenance><tns:viewUrl>http://www.trafficnz.info/camera/vie

In [61]:
string_body = String(res.body)


"<?xml version='1.0' encoding='UTF-8'?><tns:getCamerasResponse xmlns:tns=\"https://infoconnect.highwayinfo.govt.nz/schemas/camera2\"><tns:camera><tns:description>South along Hinds Highway from Lagmhor Rd</tns:description><tns:direction>Southbound</tns:direction><tns:group" ⋯ 190736 bytes ⋯ "ne><tns:region>West Coast</tns:region><tns:thumbUrl>http://www.trafficnz.info/camera/thumb/745.jpg</tns:thumbUrl><tns:underMaintenance>false</tns:underMaintenance><tns:viewUrl>http://www.trafficnz.info/camera/view/745</tns:viewUrl></tns:camera></tns:getCamerasResponse>"

In [63]:
# body of response to  xml
parsed_xml = parsexml(string_body)

EzXML.Document(EzXML.Node(<DOCUMENT_NODE@0x000000007a19f580>))

In [64]:
root = parsed_xml.root

EzXML.Node(<ELEMENT_NODE[getCamerasResponse]@0x000000007e873be0>)

In [83]:
lat_list = []
lon_list = []
imageUrl_list = []
region_list = []
direction_list = []
name_list = []
description_list = []
for desc in nodecontent.(findall("//tns:camera//tns:description/text()", root))
    push!(description_list, desc)
end

for lat in nodecontent.(findall("//tns:camera//tns:lat/text()", root))
    push!(lat_list,lat)
end

for lon in nodecontent.(findall("//tns:camera//tns:lon/text()", root))
    push!(lon_list,lon)
end

# for direction in nodecontent.(findall("//tns:camera//tns:direction/text()", root))
#      push!(direction_list,direction)
# end

for image in nodecontent.(findall("//tns:camera//tns:imageUrl/text()", root))
    push!(imageUrl_list,image)
end

for reg in nodecontent.(findall("//tns:camera//tns:region/text()", root))
    push!(region_list,reg)
end

for nam in nodecontent.(findall("//tns:camera//tns:name/text()", root))
    push!(name_list, nam)
end


# cameras_df = DataFrame(
#     #lat = lat_list,

#     #lon = lon_list,
#     imageUrl = imageUrl_list,
#     region = region_list
#     )


In [82]:
# Write to csv file
# This is write per line
csvfile = open("cameras.csv", "w") # open the IOStream
write(csvfile,"lat,lon,imageUrl,description,region\n")
# loop to write 288 rows
for i = 1:288
    cam_line = lat_list[i], lon_list[i], imageUrl_list[i], description_list[i], region_list[i]
    write(csvfile, join(cam_line,","), "\n")
end
close(csvfile) # close stream
#IOStream(<file cameras.csv)

In [84]:
# check lenths
println(length(lat_list))
println(length(lon_list))
println(length(imageUrl_list))
println(length(description_list))
#println(length(direction_list))
println(length(region_list))

288
288
288
288
321
288


In [45]:
cameras_df

,imageUrl,region
,Any,Any
1,http://www.trafficnz.info/camera/714.jpg,Ashburton
2,http://www.trafficnz.info/camera/651.jpg,Auckland
3,http://www.trafficnz.info/camera/653.jpg,Auckland
4,http://www.trafficnz.info/camera/654.jpg,Auckland
5,http://www.trafficnz.info/camera/655.jpg,Auckland
6,http://www.trafficnz.info/camera/723.jpg,Auckland
7,http://www.trafficnz.info/camera/697.jpg,Auckland
8,http://www.trafficnz.info/camera/686.jpg,Auckland
9,http://www.trafficnz.info/camera/687.jpg,Auckland


In [20]:
println(length(lat_list))
println(length(lon_list))
println(length(imageUrl_list))
println(length(id_list))

288
288
288
321


In [28]:
parse.(Float64, lon_list)
parse.(Float64, lat_list)
cameras_df = DataFrame(
    lat = lat_list,
    lon = lon_list
    #imageUrl = imageUrl_list
    )

,lat,lon
,Any,Any
1,-43.919632,171.721055
2,-36.6032731,174.6617411
3,-36.90943,174.73442
4,-36.87173,174.71018
5,-36.871997,174.704836
6,-36.685718,174.701202
7,-36.870049,174.673614
8,-36.871594,174.699344
9,-36.865869,174.666243


In [22]:
println(lat_list)
println(lon_list)

Any["-43.919632", "-36.6032731", "-36.90943", "-36.87173", "-36.871997", "-36.685718", "-36.870049", "-36.871594", "-36.865869", "-36.872107", "-36.860564", "-36.819", "-36.813", "-36.869", "-36.872", "-36.858", "-36.849", "-36.847", "-36.782", "-36.757", "-36.778", "-36.753", "-36.771", "-36.765", "-36.786", "-36.763", "-36.791", "-36.808", "-36.805", "-36.8", "-36.796", "-36.735", "-36.771763", "-36.789", "-36.799", "-36.812", "-36.658354", "-36.669183", "-37.192844", "-37.214557", "-37.230209", "-37.143496", "-36.86", "-36.997", "-37.0", "-36.878", "-36.889", "-36.915", "-36.992", "-36.957", "-37.021", "-37.028", "-37.044", "-37.053", "-37.062", "-37.085", "-37.094", "-36.899797", "-36.90074", "-36.87504", "-36.982", "-36.973", "-36.966298", "-36.996", "-36.964", "-36.984608", "-36.98", "-36.973", "-36.967669", "-36.921434017433", "-36.917", "-36.914", "-36.911", "-36.904128", "-36.928", "-36.931", "-36.941", "-36.945", "-36.953", "-36.993", "-36.998", "-36.998", "-36.996", "-36.996

In [51]:
parse.(Float64, lon_list)
parse.(Float64, lat_list)
length(lon_list)

288

In [47]:


camera_df = DataFrame(id = lat_list,
price = lon_list)

,id,price
,Any,Any
1,-43.919632,171.721055
2,-36.6032731,174.6617411
3,-36.90943,174.73442
4,-36.87173,174.71018
5,-36.871997,174.704836
6,-36.685718,174.701202
7,-36.870049,174.673614
8,-36.871594,174.699344
9,-36.865869,174.666243
